In [1]:
import numpy as np
import pandas as pd
import librosa
import pickle
import time

In [2]:
train_dir = '/Volumes/FilesHDD/CODE/mmai-proj/nsynth-train/audio/'
sample_size = 5

In [3]:
# load raw
df_train_raw = pd.read_json (path_or_buf='nsynth-train/examples.json', orient='index')

# Sample n files
df_train_sample = df_train_raw.groupby('instrument_family', as_index=False, #group by instrument family
                               group_keys=False).apply(lambda df: df.sample(sample_size)) #number of samples
# drop the synth_lead from the training dataset
df_train_sample = df_train_sample[df_train_sample['instrument_family']!=9]

In [4]:
# pickle filenames
filenames_train = df_train_sample.index.tolist()
with open('filenames_train.pickle', 'wb') as f:
    pickle.dump(filenames_train, f)

In [5]:
def feature_extract(file):
    """
    Define function that takes in a file an returns features in an array
    """
    
    print ("Extracting features for {}".format (file[len(train_dir):]))
    
    #get wave representation
    y, sr = librosa.load(file)
        
    #determine if instruemnt is harmonic or percussive by comparing means
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    if np.mean(y_harmonic)>np.mean(y_percussive):
        harmonic=1
    else:
        harmonic=0
        
    #Mel-frequency cepstral coefficients (MFCCs)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    #temporal averaging
    mfcc=np.mean(mfcc,axis=1)
    
    #get the mel-scaled spectrogram
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,fmax=8000)  
    #temporally average spectrogram
    spectrogram = np.mean(spectrogram, axis = 1)
    
    #compute chroma energy
    chroma = librosa.feature.chroma_cens(y=y, sr=sr)
    #temporally average chroma
    chroma = np.mean(chroma, axis = 1)
    
    #compute spectral contrast
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    contrast = np.mean(contrast, axis= 1)
    
    return [harmonic, mfcc, spectrogram, chroma, contrast]

In [6]:
def instrument_code(filename):
    class_names=['bass', 'brass', 'flute', 'guitar', 
             'keyboard', 'mallet', 'organ', 'reed', 
             'string', 'synth_lead', 'vocal']
    
    for name in class_names:
        if name in filename:
            return class_names.index(name)
    else:
        return None

In [7]:
start_train = time.time()

#create dictionary to store all test features
dict_train = {}
#loop over every file in the list
for file in filenames_train:
    #extract the features
    features = feature_extract(train_dir+ file + '.wav') #specify directory and .wav
    #add dictionary entry
    dict_train[file] = features

end_train=time.time()
print('Time to extract {} files is {} seconds'.format(len(filenames_train), end_train - start_train))

Time to extract 50 files is 23.390511989593506 seconds


In [13]:
print('Converting to dataframe')

#convert dict to dataframe
features_train = pd.DataFrame.from_dict(dict_train, orient='index',
                                       columns=['harmonic', 'mfcc', 'spectro', 'chroma', 'contrast'])

#extract mfccs
mfcc_train = pd.DataFrame(features_train.mfcc.values.tolist(),
                          index=features_train.index)
mfcc_train = mfcc_train.add_prefix('mfcc_')

#extract spectro
spectro_train = pd.DataFrame(features_train.spectro.values.tolist(),
                             index=features_train.index)
spectro_train = spectro_train.add_prefix('spectro_')


#extract chroma
chroma_train = pd.DataFrame(features_train.chroma.values.tolist(),
                            index=features_train.index)
chroma_train = chroma_train.add_prefix('chroma_')


#extract contrast
contrast_train = pd.DataFrame(features_train.contrast.values.tolist(),
                              index=features_train.index)
contrast_train = chroma_train.add_prefix('contrast_')

#drop the old columns
features_train = features_train.drop(labels=['mfcc', 'spectro', 'chroma', 'contrast'], axis=1)

#concatenate
df_features_train=pd.concat([features_train, mfcc_train, spectro_train, chroma_train, contrast_train],
                           axis=1, join='inner')

targets_train = []
for name in df_features_train.index.tolist():
    targets_train.append(instrument_code(name))

Converting to dataframe


In [1]:
print ('Writing to df_features.pickle')
df_features_train['targets'] = targets_train
with open('df_features.pickle', 'wb') as f:
    pickle.dump(df_features_train, f)

Writing to df_features.pickle


NameError: name 'targets_train' is not defined

In [ ]:
def instrument_code(filename):
    """
    Function that takes in a filename and returns instrument based on naming convention
    """
    class_names=['bass', 'brass', 'flute', 'guitar', 
             'keyboard', 'mallet', 'organ', 'reed', 
             'string', 'synth_lead', 'vocal']
    
    for name in class_names:
        if name in filename:
            return class_names.index(name)
    else:
        return None

targets_train = []
for name in df_features.index.tolist():
    targets_train.append(instrument_code(name))
    
df_features['targets'] = targets_train

#save the dataframe to a pickle file
with open('df_features_with_targets.pickle', 'wb') as f:
    pickle.dump(df_features, f)
    
df_features.shape